# Data 5000:  Testing and handling data

## Data Import

In [1]:
import os
import pandas as pd
buslist = []
subwaylist = []
eastlist = []
centerlist = []
dfbus= pd.DataFrame()
dfsubway= pd.DataFrame()
dfeast= pd.DataFrame()
dfcenter= pd.DataFrame()
for root, dirs, files in os.walk("."):
    for file in files: #matching folder
        st = os.path.join(root,file)
        if st[-5:]==".xlsx" and "\\bus_data\\" in st:
            buslist.append(st)
        elif st[-5:]==".xlsx" and "\\subway_data\\" in st:
            subwaylist.append(st)
        elif st[-4:]==".csv" and "\\East (Airport)\\" in st:
            eastlist.append(st)
        elif st[-4:]==".csv" and "\\City Centre\\" in st:
            centerlist.append(st)
for f in buslist:
    dfbus=pd.concat([dfbus,pd.concat(pd.read_excel(f, engine='openpyxl', index_col=None, sheet_name=None),sort=False)],sort=False)
for f in subwaylist:
    dfsubway=pd.concat([dfsubway,pd.read_excel(f, engine='openpyxl', index_col=None, sheet_name=None)],sort=False)
for f in eastlist:
    dfeast=pd.concat([dfeast,pd.read_csv(f, index_col=None)],sort=False)
for f in centerlist:
    dfcenter=pd.concat([dfcenter,pd.read_csv(f, index_col=None)],sort=False)

TypeError: cannot concatenate object of type '<class 'collections.OrderedDict'>'; only Series and DataFrame objs are valid

### Saving dataframes

In [ ]:
dfbus.to_csv("{}\\busdata.csv".format(os.getcwd()))
dfsubway.to_csv("{}\\subwaydata.csv".format(os.getcwd()))
dfeast.to_csv("{}\\eastdata.csv".format(os.getcwd()))
dfcenter.to_csv("{}\\buscenter.csv".format(os.getcwd()))

## Data cleaning

In [65]:
dfbus.loc[dfbus["Date"].isna(),"Date"]=dfbus[dfbus["Date"].isna()]["Report Date"]
dfbus.loc[dfbus["Delay"].isna(),"Delay"]=dfbus[dfbus["Delay"].isna()]["Min Delay"]
dfbus.loc[dfbus["Gap"].isna(),"Gap"]=dfbus[dfbus["Gap"].isna()]["Min Gap"]
dfbus.loc[dfbus["Route"].isna(),"Route"]=dfbus[dfbus["Route"].isna()]["Line"]
dfbus.loc[dfbus["Direction"].isna(),"Direction"]=dfbus[dfbus["Direction"].isna()]["Bound"]
dfbus.drop("Report Date", axis=1, inplace=True)
dfbus.drop("Min Delay", axis=1, inplace=True)
dfbus.drop("Min Gap", axis=1, inplace=True)
dfbus.drop("Line", axis=1, inplace=True)
dfbus.drop("Bound", axis=1, inplace=True)
dfbus = dfbus.drop(dfbus[dfbus["Date"].isna()].index)

dfbus.reset_index(inplace=True)
dfbus.drop("Incident ID", axis=1, inplace=True)

KeyError: 'Report Date'

In [100]:
dfbus[dfbus["Incident ID"].isna()]

,level_0,level_1,Route,Time,Day,Location,Incident,Direction,Vehicle,Min Delay,Incident ID,Delay,Gap,Date
0,Jan 2014,0,95,00:23:00,Wednesday,York Mills station,Mechanical,E,1734.0,NaN,NaN,10.0,20.0,2014-01-01
1,Jan 2014,1,102,00:55:00,Wednesday,Entire run for route,General Delay,b/w,8110.0,NaN,NaN,33.0,66.0,2014-01-01
2,Jan 2014,2,54,01:28:00,Wednesday,lawrence and Warden,Mechanical,WB,7478.0,NaN,NaN,10.0,20.0,2014-01-01
3,Jan 2014,3,112,01:30:00,Wednesday,Kipling Station,Emergency Services,N,8084.0,NaN,NaN,18.0,36.0,2014-01-01
4,Jan 2014,4,24,01:37:00,Wednesday,VP and Ellesmere,Investigation,n,7843.0,NaN,NaN,10.0,20.0,2014-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517339,Aug 21,1224,510,21:10,Tuesday,SPADINA AND HARBORD,Mechanical,S,4523.0,NaN,NaN,14.0,23.0,2021-08-31
517340,Aug 21,1225,506,21:31,Tuesday,COLLEGE AND LANSDOWNE,Operations,E,4503.0,NaN,NaN,9.0,17.0,2021-08-31
517341,Aug 21,1226,501,21:55,Tuesday,QUEEN AND CONNAUGHT,Operations,E,4403.0,NaN,NaN,10.0,20.0,2021-08-31
517342,Aug 21,1227,506,23:36,Tuesday,CARLTON AND JARVIS,Held By,E,4569.0,NaN,NaN,75.0,84.0,2021-08-31


In [104]:
set(dfbus["Location"])

110567

In [107]:
coordinateDictionary= {}
for location in set(dfbus["Location"]):
    coordinateDictionary[location] = getCoordinates(location)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Avenue+rd+and+401%2C+Toronto&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [91]:
dfbus.reset_index(inplace=True)

In [101]:
dfbus.drop("Incident ID", axis=1, inplace=True)

In [106]:
from geopy.geocoders import Nominatim
location = "Entire Route"

def getCoordinates(location):
    loc = Nominatim(user_agent="GetLoc")
    getLoc = loc.geocode("{}, Toronto".format(location))
    if (getLoc is None):
        return (None, None)
    else:
        return (getLoc.latitude,getLoc.longitude)
print(getCoordinates(location))

(None, None)
